# 타이타닉 생존자 예측

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
df=sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


## 1. 데이터 전처리
  * Feature selection
  * Feature filtering

In [3]:
df=df[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked', 'deck']]
print(df.shape)
df.head()

(891, 9)


,survived,pclass,sex,age,sibsp,parch,fare,embarked,deck
0,0,3,male,22.0,1,0,7.2500,S,NaN
1,1,1,female,38.0,1,0,71.2833,C,C
2,1,3,female,26.0,0,0,7.9250,S,NaN
3,1,1,female,35.0,1,0,53.1000,S,C
4,0,3,male,35.0,0,0,8.0500,S,NaN


- 결측치 처리

In [4]:
df.isna().sum()

survived      0
pclass        0
sex           0
age         177
sibsp         0
parch         0
fare          0
embarked      2
deck        688
dtype: int64

In [5]:
# age => 평균값으로 대체
df.age.fillna(df.age.mean(), inplace=True)
df.age.isnull().sum()

0

In [6]:
# embarked - 최빈값으로 대체
df.embarked.value_counts()  # 최빈값을 확인하는 코드

S    644
C    168
Q     77
Name: embarked, dtype: int64

In [7]:
df.embarked.fillna('S', inplace=True)
df.embarked.isna().sum()

0

In [10]:
# deck - 열 삭제(너무 많은 데이터가 결측치여서 의미가 없음)
df.drop(columns=['deck'], inplace=True)


KeyError: ignored

In [11]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [16]:
df.sex=le.fit_transform(df.sex)
df.embarked=le.fit_transform(df.embarked)
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


## 2. Train/Test dataset으로 분리

In [17]:
# X와 y를 넘파이 배열으로
X=df.iloc[:,1:].values
y=df.survived.values
X.shape, y.shape

((891, 7), (891,))

In [18]:
# y 값의 분포
np.unique(y, return_counts=True)

(array([0, 1]), array([549, 342]))

In [19]:
# train/test dataset 만들기
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=2022
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((712, 7), (179, 7), (712,), (179,))

In [20]:
np.unique(y_train, return_counts=True)

(array([0, 1]), array([439, 273]))

## 3. Random Forest 모델을 이용한 분석
  * 의사결정 트리를 이용해 분석하기 때문에 Forest 모델 + 데이터를 복원추출할때 랜덤으로 추출하므로 Random이라는 이름이 붙음

In [21]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(random_state=2022)
rfc.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 2022,
 'verbose': 0,
 'warm_start': False}

In [22]:
rfc.fit(X_train, y_train)

RandomForestClassifier(random_state=2022)

## 4. 모델 예측 및 평가

In [23]:
rfc.score(X_test, y_test)  # 사실 얘는 과적합되어있음

0.8324022346368715

## 5. GridSearchCV를 이용한 최적의 hyperparameter 찾기

In [26]:
params={
    'max_depth' : [2,4,6,8],
    'min_samples_split': [2,4,6,8,10]
}

In [29]:
from sklearn.model_selection import GridSearchCV
grid_rf=GridSearchCV(rfc, params, scoring='accuracy', cv=5)
grid_rf.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=2022),
             param_grid={'max_depth': [2, 4, 6, 8],
                         'min_samples_split': [2, 4, 6, 8, 10]},
             scoring='accuracy')

In [30]:
grid_rf.best_params_

{'max_depth': 4, 'min_samples_split': 4}

In [32]:
params={
    'max_depth' : [3,4,5],
    'min_samples_split': [3,4,5]
}
grid_rf=GridSearchCV(rfc, params, scoring='accuracy', cv=5)
%time grid_rf.fit(X_train, y_train)

CPU times: user 8.5 s, sys: 47 ms, total: 8.54 s
Wall time: 9.36 s


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=2022),
             param_grid={'max_depth': [3, 4, 5],
                         'min_samples_split': [3, 4, 5]},
             scoring='accuracy')

In [33]:
grid_rf.best_params_

{'max_depth': 4, 'min_samples_split': 3}

In [35]:
best_rf=grid_rf.best_estimator_
best_rf.score(X_test, y_test)

0.8212290502793296

## 6. 테스트 데이터에 적용

In [36]:
X_test[25], y_test[25]

(array([ 3.  ,  1.  , 45.  ,  0.  ,  0.  ,  8.05,  2.  ]), 1)

In [38]:
best_rf.predict( X_test[25].reshape(1,-1) )

array([0])

## 7.비교를 위해 만드는 엉터리 분류기

In [45]:
# 여성의 생존률?
df.groupby(['sex', 'pclass'])[['survived']].mean()

survived
sex pclass          
0   1       0.968085
    2       0.921053
    3       0.500000
1   1       0.368852
    2       0.157407
    3       0.135447

In [59]:
from sklearn.base import BaseEstimator

# 밑에 코드처럼 상속을 받기 위해서는 괄호 안에 부모 Class를 넣어줌!
class MyClassifier(BaseEstimator):   # class명은 대문자로 시작하는게 일반적임
  # fit(), predict() method만 재정의(Overriding) 해보자
  def fit(self, X, y):      # method 정의도 함수 정의와 비슷하지만 괄호 안에 self를 넣어주는게 매우 중요
    pass  # 어차피 엉터리 예측이므로 fitting은 이뤄지지 않음. continue와 똑같은 코드
  def predict(self, X):
    pred=np.zeros(X.shape[0], int)    # X의 행의 갯수와 동일한 길이를 가지며, 0으로 초기화된 배열 생성
    for i in range(X.shape[0]):
      if X[i, 1]==0:    # 여성이면
        pred[i]=1    # 생존
      else:    # 남성이면
        pred[i]=0    # 사망
    return pred

In [60]:
my_clf=MyClassifier()
my_clf.fit(X_train, y_train)
pred_my=my_clf.predict(X_test)


In [61]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred_my)   # 엉터리 모델의 정확도도 생각보다 매우 높음을 알 수 있음

0.7877094972067039

- 결론 :  모델의 성능을 평가할 때 무조건적으로 정확도만을 사용하는 것은 지양해야 함!



In [64]:
pred_rf=best_rf.predict(X_test)
score_df=pd.DataFrame({'true_class' : y_test, 'predicted_class' : pred_rf, 'my_predict' : pred_my})
score_df.head()

,true_class,predicted_class,my_predict
0,1,0,0
1,0,0,0
2,1,0,1
3,0,0,0
4,0,0,0


In [65]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, pred_rf)  # confusion_matrix(실제 데이터, 예측 데이터)

array([[103,   7],
       [ 25,  44]])

In [66]:
confusion_matrix(y_test, pred_my)

array([[96, 14],
       [24, 45]])

In [67]:
from sklearn.metrics import precision_score, recall_score
print(f"정밀도(precision) : {precision_score(y_test, pred_rf)}, 재현율(recall) : {recall_score(y_test, pred_rf)}")

정밀도(precision) : 0.8627450980392157, 재현율(recall) : 0.6376811594202898


In [70]:
print(f"정밀도(precision) : {precision_score(y_test, pred_my)}, 재현율(recall) : {recall_score(y_test, pred_my)}")

정밀도(precision) : 0.7627118644067796, 재현율(recall) : 0.6521739130434783


In [71]:
# F1 Score(정밀도와 재현율의 조화평균)
from sklearn.metrics import f1_score
f1_score(y_test, pred_rf), f1_score(y_test, pred_my)

(0.7333333333333333, 0.703125)

In [72]:
# AUC Score
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, pred_rf), roc_auc_score(y_test, pred_my)

(0.787022397891963, 0.7624505928853755)